In [1]:
import pulp
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dataclasses import dataclass, field
from typing import List, Dict
from datetime import date, timedelta, datetime
import json

# Le decimos a Plotly que el renderizador por defecto es el navegador
pio.renderers.default = "browser"

# --- CAMBIO 1: Nueva función para manejar fechas laborales ---
def calcular_fecha_laboral(fecha_inicio, dias_laborables_a_sumar, dias_laborables_semana):
    """
    Calcula la fecha final sumando solo días laborables.
    """
    fecha_actual = fecha_inicio
    dias_sumados = 0
    while dias_sumados < dias_laborables_a_sumar:
        # El día 0 es Lunes. weekday() devuelve 0 para Lunes, 1 para Martes...
        if fecha_actual.strftime('%A') in dias_laborables_semana:
            dias_sumados += 1
        if dias_sumados < dias_laborables_a_sumar:
            fecha_actual += timedelta(days=1)
    return fecha_actual

# 1. DEFINICIÓN DE ESTRUCTURAS DE DATOS
@dataclass
class Resource:
    name: str
    availability: Dict[str, float] = field(default_factory=dict)

@dataclass
class Task:
    id: str
    name: str
    project_id: str
    hours: float
    sequence: int
    compatible_resources: List[str] = field(default_factory=list)
    subcontractable: bool = False
    requires_client_validation: bool = False
    validation_delay_days: int = 0
    predecessors: List[str] = field(default_factory=list)

@dataclass
class Project:
    id: str
    name: str

# 2. CLASE DEL OPTIMIZADOR CON PULP
class OptimizadorProyectos:
    def __init__(self, proyectos, tareas, recursos, params):
        self.proyectos = proyectos
        self.tareas = tareas
        self.recursos = recursos
        self.params = params
        self.task_map = {t.id: t for t in self.tareas}
        self.project_map = {p.id: p for p in self.proyectos}
        self.model = pulp.LpProblem("Maximizacion_Proyectos", pulp.LpMaximize)
        self.vars = {}

    @classmethod
    def desde_json(cls, ruta_fichero: str):
        print(f"Cargando instancia desde: {ruta_fichero}")
        with open(ruta_fichero, 'r') as f:
            data = json.load(f)
        proyectos = [Project(**p) for p in data['proyectos']]
        tareas = [Task(**t) for t in data['tareas']]
        recursos = [Resource(**r) for r in data['recursos']]
        params = data['parametros']
        params['fecha_inicio'] = datetime.strptime(params['fecha_inicio'], '%Y-%m-%d').date()
        return cls(proyectos, tareas, recursos, params)

    def construir_modelo(self):
        """Traduce el modelo matemático a código PuLP con penalización por duración."""
        H = self.params["horizonte_dias"]
        DIAS = list(range(1, H + 1))
        
        # --- Variables ---
        self.vars['x'] = pulp.LpVariable.dicts("horas", ((t.id, r.name, d) for t in self.tareas for r in self.recursos for d in DIAS), lowBound=0, cat='Continuous')
        self.vars['s'] = pulp.LpVariable.dicts("inicio_tarea", (t.id for t in self.tareas), lowBound=1, upBound=H, cat='Integer')
        self.vars['e'] = pulp.LpVariable.dicts("fin_tarea", (t.id for t in self.tareas), lowBound=1, upBound=H, cat='Integer')
        self.vars['SubT'] = pulp.LpVariable.dicts("subcontrata_tarea", (t.id for t in self.tareas), cat='Binary')
        self.vars['C'] = pulp.LpVariable.dicts("completa_proyecto", (p.id for p in self.proyectos), cat='Binary')
        self.vars['SubP'] = pulp.LpVariable.dicts("subcontrata_proyecto", (p.id for p in self.proyectos), cat='Binary')
        self.vars['CI'] = pulp.LpVariable.dicts("completa_interno", (p.id for p in self.proyectos), cat='Binary')
        self.vars['CS'] = pulp.LpVariable.dicts("completa_subcontratado", (p.id for p in self.proyectos), cat='Binary')
        # --- CAMBIO 2: Variable para la duración de la tarea ---
        self.vars['duracion'] = pulp.LpVariable.dicts("duracion_tarea", (t.id for t in self.tareas), lowBound=0, cat='Integer')

        # --- Objetivo ---
        alpha = self.params['alpha']
        beta = self.params['beta']
        gamma = 0.1  # Factor de penalización por duración
        
        beneficio_proyectos = pulp.lpSum(alpha * self.vars['CI'][p.id] + beta * self.vars['CS'][p.id] for p in self.proyectos)
        penalizacion_duracion = gamma * pulp.lpSum(self.vars['duracion'][t.id] for t in self.tareas)
        
        self.model += beneficio_proyectos - penalizacion_duracion, "Beneficio_Neto_Con_Penalizacion"

        # --- Restricciones ---
        M = H * 10
        for t in self.tareas:
            z_td = pulp.LpVariable.dicts(f"activa_{t.id}", DIAS, cat='Binary')
            
            # --- CAMBIO 2 (cont.): Definir la duración de la tarea ---
            self.model += self.vars['duracion'][t.id] == self.vars['e'][t.id] - self.vars['s'][t.id] + 1, f"Def_Duracion_{t.id}"

            self.model += pulp.lpSum(self.vars['x'][t.id, r.name, d] for r in self.recursos if r.name in t.compatible_resources for d in DIAS) == t.hours * (1 - self.vars['SubT'][t.id]), f"Horas_{t.id}"
            if not t.subcontractable:
                self.model += self.vars['SubT'][t.id] == 0, f"No_Sub_{t.id}"

            for d in DIAS:
                self.model += pulp.lpSum(self.vars['x'][t.id, r.name, d] for r in self.recursos) <= M * z_td[d], f"Link_activa_{t.id}_{d}"
                self.model += self.vars['e'][t.id] >= d * z_td[d], f"Def_Fin_{t.id}_{d}"
                self.model += self.vars['s'][t.id] <= d + M * (1 - z_td[d]), f"Def_Inicio_{t.id}_{d}"
            self.model += self.vars['s'][t.id] <= self.vars['e'][t.id], f"Inicio_antes_de_Fin_{t.id}"

            for pred_id in t.predecessors:
                pred_task = self.task_map[pred_id]
                retraso_validacion = pred_task.validation_delay_days if pred_task.requires_client_validation else 0
                self.model += self.vars['s'][t.id] >= self.vars['e'][pred_id] + 1 + retraso_validacion, f"Secuencia_{pred_id}_{t.id}"

        for r in self.recursos:
            for d in DIAS:
                # El mapeo a día de la semana ahora solo se usa aquí, no en la visualización
                dia_semana_idx = (self.params['fecha_inicio'] + timedelta(days=d-1)).weekday()
                if dia_semana_idx < len(self.params["dias_laborables"]):
                    dia_semana_nombre = self.params["dias_laborables"][dia_semana_idx]
                    disponibilidad = r.availability.get(dia_semana_nombre, 0)
                else:
                    disponibilidad = 0 # Fin de semana
                self.model += pulp.lpSum(self.vars['x'][t.id, r.name, d] for t in self.tareas) <= disponibilidad, f"Disp_{r.name}_{d}"
        
        for p in self.proyectos:
            tareas_del_proyecto = [t for t in self.tareas if t.project_id == p.id]
            if tareas_del_proyecto:
                for t in tareas_del_proyecto:
                    self.model += self.vars['e'][t.id] - M * self.vars['SubT'][t.id] <= H + M * (1 - self.vars['C'][p.id]), f"Completitud_{p.id}_{t.id}"
                self.model += pulp.lpSum(self.vars['SubT'][t.id] for t in tareas_del_proyecto) <= M * self.vars['SubP'][p.id], f"Define_SubP_upper_{p.id}"
                for t in tareas_del_proyecto:
                    self.model += self.vars['SubP'][p.id] >= self.vars['SubT'][t.id], f"Define_SubP_lower_{p.id}_{t.id}"
            self.model += self.vars['CI'][p.id] <= self.vars['C'][p.id], f"LinCI1_{p.id}"
            self.model += self.vars['CI'][p.id] <= 1 - self.vars['SubP'][p.id], f"LinCI2_{p.id}"
            self.model += self.vars['CI'][p.id] >= self.vars['C'][p.id] - self.vars['SubP'][p.id], f"LinCI3_{p.id}"
            self.model += self.vars['CS'][p.id] <= self.vars['C'][p.id], f"LinCS1_{p.id}"
            self.model += self.vars['CS'][p.id] <= self.vars['SubP'][p.id], f"LinCS2_{p.id}"
            self.model += self.vars['CS'][p.id] >= self.vars['C'][p.id] + self.vars['SubP'][p.id] - 1, f"LinCS3_{p.id}"

    def resolver(self, solver=None):
        self.model.solve(solver)
        return pulp.LpStatus[self.model.status]

    def mostrar_resultados(self):
        # (Sin cambios aquí)
        print("\n--- RESULTADOS DE LA OPTIMIZACIÓN ---")
        print(f"Estado: {pulp.LpStatus[self.model.status]}")
        print(f"Beneficio Neto Óptimo: {pulp.value(self.model.objective):.2f}")
        print("\n**Estado de los Proyectos:**")
        for p in self.proyectos:
            if pulp.value(self.vars['C'][p.id]) == 1:
                tipo = "Subcontratado" if pulp.value(self.vars['SubP'][p.id]) == 1 else "Interno"
                print(f"- Proyecto '{p.name}': COMPLETADO ({tipo})")
            else:
                print(f"- Proyecto '{p.name}': NO COMPLETADO")
        print("\n**Detalle de Tareas:**")
        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 1:
                print(f"- Tarea '{t.name}' ({self.task_map[t.id].project_id}): SUBCONTRATADA")
            else:
                inicio = pulp.value(self.vars['s'][t.id])
                fin = pulp.value(self.vars['e'][t.id])
                duracion = pulp.value(self.vars['duracion'][t.id])
                print(f"- Tarea '{t.name}' ({self.task_map[t.id].project_id}): Inicia día {inicio:.0f}, Finaliza día {fin:.0f} (Duración: {duracion:.0f} días)")
        print("\n**Asignación de Horas (resumen):**")
        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 0:
                for r in self.recursos:
                    horas_asignadas = sum(pulp.value(self.vars['x'][t.id, r.name, d]) for d in range(1, self.params['horizonte_dias'] + 1))
                    if horas_asignadas > 0.1:
                        print(f"  - Recurso '{r.name}' trabaja {horas_asignadas:.1f}h en la tarea '{t.name}'")
    
    def visualizar_gantt(self):
        if self.model.status != pulp.LpStatusOptimal: return
        fecha_inicio = self.params["fecha_inicio"]
        dias_lab = self.params["dias_laborables"]
        gantt_data = []

        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 0:
                start_day = int(pulp.value(self.vars['s'][t.id]))
                finish_day = int(pulp.value(self.vars['e'][t.id]))
                
                # --- CAMBIO 1 (cont.): Usar la nueva función de fechas ---
                start_date = calcular_fecha_laboral(fecha_inicio, start_day - 1, dias_lab)
                finish_date = calcular_fecha_laboral(fecha_inicio, finish_day, dias_lab) # No restamos 1 para que sea inclusivo
                
                assigned_resource = "N/A"
                for r in self.recursos:
                    if sum(pulp.value(self.vars['x'][t.id, r.name, d]) for d in range(1, self.params['horizonte_dias'] + 1)) > 0.1:
                        assigned_resource = r.name
                        break
                
                gantt_data.append(dict(Task=f"{self.project_map[t.project_id].name}: {t.name}", Start=start_date, Finish=finish_date, Project=self.project_map[t.project_id].name, Resource=assigned_resource))
        
        if not gantt_data: print("\nNo hay tareas internas para mostrar en el Gantt."); return
        df = pd.DataFrame(gantt_data)
        fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Project", title="Diagrama de Gantt de la Planificación", hover_data=["Resource"])
        fig.update_yaxes(categoryorder="total ascending")
        fig.show()

    def visualizar_carga_diaria(self):
        if self.model.status != pulp.LpStatusOptimal: return
        fecha_inicio = self.params["fecha_inicio"]
        dias_lab = self.params["dias_laborables"]
        horas_data = []

        for (t_id, r_name, d), var in self.vars['x'].items():
            if var.varValue is not None and var.varValue > 0.1:
                task = self.task_map[t_id]
                # --- CAMBIO 1 (cont.): Usar la nueva función de fechas ---
                fecha_real = calcular_fecha_laboral(fecha_inicio, d-1, dias_lab)
                horas_data.append({"Fecha": fecha_real, "Recurso": r_name, "Horas": var.varValue, "Tarea": task.name, "Proyecto": self.project_map[task.project_id].name})
        
        if not horas_data: print("\nNo se asignaron horas a ningún recurso."); return
        df = pd.DataFrame(horas_data)
        
        for recurso in sorted(df['Recurso'].unique()):
            df_recurso = df[df['Recurso'] == recurso]
            fig = px.bar(df_recurso, x='Fecha', y='Horas', color='Tarea', title=f"Carga Diaria para: {recurso}", hover_data=["Proyecto", "Horas"])
            fig.update_layout(yaxis_range=[0,10])
            fig.show()

In [ ]:

fichero_instancia = r'..\instancias\instancia_real.json'
optimizador = OptimizadorProyectos.desde_json(fichero_instancia)
print("Construyendo el modelo de optimización...")
optimizador.construir_modelo()
print("Resolviendo...")
estado = optimizador.resolver()
if estado == 'Optimal':
    optimizador.mostrar_resultados()
    print("\nGenerando diagrama de Gantt...")
    optimizador.visualizar_gantt()
    print("\nGenerando gráficos de carga diaria por recurso...")
    optimizador.visualizar_carga_diaria()
else:
    print(f"\nNo se encontró una solución óptima. Estado: {estado}")

<>:1: SyntaxWarning:

invalid escape sequence '\i'

<>:1: SyntaxWarning:

invalid escape sequence '\i'

C:\Users\USER\AppData\Local\Temp\ipykernel_71236\1025307049.py:1: SyntaxWarning:

invalid escape sequence '\i'



Cargando instancia desde: ..\instancias\instancia_real.json
Construyendo el modelo de optimización...
Resolviendo...
